# Lesson3 系列データで分類・予測させてみよう（RNN, LSTM）

## 目次

- 3.4の解答
- Section4 実装②
    - 4.0 データの用意
    - 4.1 モデル構築
    - 4.2 モデルの学習
    - 4.3 モデルによる予測
    - 4.4 モデルの可視化

## Section3 Checkクイズの解答

問題1: 2, 問題2: 3, 問題3: 1, 問題4: 4

## Section4 実装②

ここでは、1949年1月から1960年12月までの月次航空会社乗客数のデータをLSTMで学習し、予測を行ってみましょう。

データは次のリンクのページのものを用います。

https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line

このデータセットには、1列目に年月、2列目にその月の航空会社乗客数が千人単位で記録されています。

### 4.0 データの用意

**ここで利用するデータセットはリポジトリに含まれていません。**

上記リンク先のページ左上段にある "Export" をクリックの上、"Exports"->"Download data"->"CSV (,)" を選択していただき、

ご自身でダウンロードして頂くようお願い致します。

ダウンロードしたデータセットはdataフォルダ内に配置していただく必要があります。

データセットの配置が完了したら、まずは配置したデータセットの読み込みを行いましょう。

今回は2列目の乗客数のみ読み込みます。なお、データセットがcsv形式なのでpandas.read_csvを用いてロードを行います。

In [ ]:
%matplotlib inline

import pandas as pd

# データの読み込み（データセット中のYのみロード）
dataframe = pd.read_csv('data/international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values.astype('float32')

データが人数という単位を持っているので、この単位への依存性をなくすため（また予測性能向上のため）スケーリングを行います。

なお、スケーリングには、ライブラリscikit-learnの`MinMaxScaler`クラスを使用します。

`MinMaxScaler`は初期化時に`feature_range`引数として、どの範囲にスケーリングするかを指定すると、

`fit_trasform`メソッドでデータセットをスケーリングし、またスケーリングに用いたパラメータを記憶します。

そのため逆変換も可能で、`inverse_transform`メソッドを用いれば、データのスケーリングを元に戻すことができます。（後ほど結果の処理で使用）

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# [0,1]に正規化
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

続いて、データセットを訓練用とテスト用に分割します。時系列の予測が目的のため、ある時点を境に訓練用、テスト用と分割します。

In [ ]:
# 訓練データとテストデータを分割（時系列の予測が目的のため、ある時点から先をテストデータとする）
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

時系列の予測の場合、ある時点のデータのみ参照して同じ時点の予測を行うことは困難です。

そこで、数回前のデータまで参照できるようにしましょう。RNNでは、この数回前までのデータを1系列とみなして入力に使用します。

In [ ]:
import numpy as np

# X=[data[t-look_back],...,data[t-1]], Y=data[t]となるデータセットに変換
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# [samples, time steps, features]へ変形
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

### 4.1 モデル構築

ここでは、先程紹介したLSTM（4ユニット）を使用してモデルを構築します。

また、最終的に必要な出力はスカラー値なので、最後にDenseレイヤーを使用し、損失関数はMSEを用います。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM

# LSTMにDenseを接続し、数値を予測（MSEで評価）
model = Sequential()

model.add(LSTM(4, input_shape=(look_back, 1))) # input_shape=(系列長T, x_tの次元), output_shape=(units,)
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

### 4.2 モデルの学習

In [ ]:
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

### 4.3 モデルによる予測

予測の精度をRMSEで評価してみましょう。ここではイメージが湧くように、正規化を解いて人数の単位で出力しています。

In [ ]:
import math
from sklearn.metrics import mean_squared_error

# テストデータに対する予測（評価のため訓練データも）
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# 正規化を元に戻す
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# 平均二乗誤差のルートで評価
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train RMSE: %.2f' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test RMSE: %.2f' % (testScore))

最後に、予測データのプロットを行って視覚的に評価を行いましょう。

In [ ]:
import matplotlib.pyplot as plt

# 訓練データ部分の予測データ
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# テストデータ部分の予測データ
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

plt.plot(scaler.inverse_transform(dataset), label='Dataset')
plt.plot(trainPredictPlot, label='Prediction under train data')
plt.plot(testPredictPlot, label='Prediction under test data')
plt.legend()
plt.show()

### 4.4 モデルの可視化

In [ ]:
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))